In [154]:
import sklearn
from efficientnet.keras import preprocess_input
import os
import cv2
import librosa
import librosa.display
from lime.lime_tabular import LimeTabularExplainer
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.callbacks import ModelCheckpoint
from datetime import datetime 
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.python.keras.optimizers import adam_v2
from sklearn import metrics
from tensorflow.python.keras.callbacks import ModelCheckpoint
from datetime import datetime 

import tensorflow.compat.v1.keras.backend as K 
import tensorflow as tf 
tf.compat.v1.disable_eager_execution()

In [119]:
#get the data path
trainpath= ("./reduce_sound/")

# get the training classes
keys=os.listdir(trainpath)
values=range(len(keys))

training_classes = np.array(keys)
print('training_classes = ',training_classes)

training_classes =  ['grey-cheeked warbler' 'grey-sided laughingthrush'
 'grey-winged blackbird' 'himalayan cuckoo' 'indian spot-billed duck'
 'mountain scops owl' 'mountain tailorbird' 'pale blue flycatcher'
 'scaly laughingthrush']


In [120]:
#Loading training data
   
X_train = []
y_train = []
train_label=[]
folder_item_numbers = []
total_files=0
for folder in  os.listdir(trainpath ) : 
    train_label.append(folder)
    files = glob(pathname= str( trainpath  + folder + '/*.wav'))
    folder_item_numbers.append(len(files))
    total_files= total_files+len(files)
print('--------------------------------------------------')        
foldernames=pd.DataFrame({'Folder_name':train_label})
itemnumbers=pd.DataFrame({'Number of Files':folder_item_numbers})
informations=pd.concat([foldernames,itemnumbers],axis=1)
print(informations)
print('--------------------------------------------------')  
print('Totall Number of Files:                   {}'.format(total_files))    

--------------------------------------------------
                 Folder_name  Number of Files
0       grey-cheeked warbler              150
1  grey-sided laughingthrush              162
2      grey-winged blackbird              150
3           himalayan cuckoo              168
4    indian spot-billed duck              156
5         mountain scops owl              120
6        mountain tailorbird              144
7       pale blue flycatcher              168
8       scaly laughingthrush              132
--------------------------------------------------
Totall Number of Files:                   1350


In [121]:
def extract(file):
    audio, sample_rate = librosa.load(file) 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    #print(mfccs_scaled_features)
    return mfccs_scaled_features

In [122]:
extracted_features=[]
final_df=[]
row=0
for folder in os.listdir(trainpath):
    audio= glob(trainpath+folder+"/*.wav")
    for i in range(len(audio)):
        s=os.path.join(os.path.abspath(audio[i]))
        feature=extract(s)
        print('\nExtracting Feature: {} / {} Files of the classs: {}\nPath: {}'.format(i,len(audio),folder, audio[i]))
        extracted_features.append([feature,folder])
        row_feature = []
        for i in range (len(feature)):
            row_feature.append(feature[i])
        row_feature.append(folder)
        print('\nRow added {} \n '.format(len(final_df)))
        row = row+1
        print(row_feature)
        final_df.append(row_feature)
                        


 Extracting Feature: 0 / 150 Files of the classs: grey-cheeked warbler
Path: ./reduce_sound/grey-cheeked warbler\phylloscopus poliogenys0.wav

Row added 0 
 
[-472.10077, 58.775963, 7.6819854, 23.957186, 0.5927175, 16.293337, 2.3274417, 7.689467, 4.08543, 5.579339, 0.87876517, 7.2429514, 1.0136265, 5.158349, 0.9515722, 4.718149, -1.6044178, 6.2166142, -1.5918176, 5.3788652, -0.5040119, 3.7143116, 0.35114777, 3.4020994, -0.4436755, 4.263005, -1.5043738, 4.3749285, -1.5044032, 3.7154527, -0.42449802, 3.295277, -1.2969853, 2.8804922, -1.4427377, 2.6795812, -0.6610387, 2.3002868, -0.9078946, 2.5063524, 'grey-cheeked warbler']

 Extracting Feature: 1 / 150 Files of the classs: grey-cheeked warbler
Path: ./reduce_sound/grey-cheeked warbler\phylloscopus poliogenys1.wav

Row added 1 
 
[-626.49115, 12.133249, -13.319744, -1.6208665, -5.934765, -5.031613, -9.316302, 11.564653, -13.300512, 1.906825, 1.9348176, -3.937581, 0.26717025, -2.0396464, 3.532683, -4.632345, 0.7469176, -0.033046957, -3.2


 Extracting Feature: 13 / 150 Files of the classs: grey-cheeked warbler
Path: ./reduce_sound/grey-cheeked warbler\phylloscopus poliogenys11.wav

Row added 13 
 
[-334.58188, 22.666418, -22.608894, 36.052376, -23.55992, 46.568264, 5.394583, -0.39557633, 16.99482, -6.4631743, 12.5293045, 13.029734, 3.3593209, 6.7859926, 9.365678, 11.619059, -4.5098023, 15.918698, -1.5755117, 12.159097, 1.0701596, 11.869171, 3.5575302, 7.600545, -1.8335422, 12.614889, 0.044040978, 6.0260606, 3.4068162, 6.7545505, -0.081250295, 3.3225849, 5.3728666, 5.1541433, 0.6990079, 4.4419465, 4.558367, 2.3744001, 2.1143038, 6.6105566, 'grey-cheeked warbler']

 Extracting Feature: 14 / 150 Files of the classs: grey-cheeked warbler
Path: ./reduce_sound/grey-cheeked warbler\phylloscopus poliogenys110.wav

Row added 14 
 
[-643.472, -12.019425, 2.4922085, 10.8078575, -8.522024, 8.385024, -0.9295703, 2.000164, 2.0341558, 0.5777646, 0.12832858, 1.983705, -0.53317076, 0.33428127, 2.9475358, -3.5445757, 3.4918578, -0.110932


 Extracting Feature: 26 / 150 Files of the classs: grey-cheeked warbler
Path: ./reduce_sound/grey-cheeked warbler\phylloscopus poliogenys121.wav

Row added 26 
 
[-668.2224, -24.88135, -1.681118, 22.017164, -22.576536, 13.171788, -1.097335, -4.235355, 4.4152017, -2.356859, 1.9531435, -2.736311, 2.029313, 0.8246106, -0.91415614, 0.60644233, -1.0159457, 2.252586, -2.584998, 1.8928386, -0.037994187, -1.1031736, 1.3944252, -0.8564152, 0.43377465, 0.15895163, -0.56539816, 1.042165, -1.0637535, 1.059601, -0.89993393, 0.585826, -0.15861031, 0.07797005, -0.005204304, 0.013963708, 0.13448937, 0.109452665, -0.28063074, 0.6878851, 'grey-cheeked warbler']

 Extracting Feature: 27 / 150 Files of the classs: grey-cheeked warbler
Path: ./reduce_sound/grey-cheeked warbler\phylloscopus poliogenys122.wav

Row added 27 
 
[-642.87866, -9.666618, 2.883715, 9.441191, -12.107383, 9.034517, -3.635299, -2.4748154, 3.9414093, -3.6246333, 3.011409, -0.20296125, 0.91008455, 1.3412495, -0.05982918, 1.3830955, -0


 Extracting Feature: 39 / 150 Files of the classs: grey-cheeked warbler
Path: ./reduce_sound/grey-cheeked warbler\phylloscopus poliogenys133.wav

Row added 39 
 
[-690.8224, 0.825905, 6.3423095, 6.2125773, -9.939589, 4.0914583, -4.2422624, 1.3222572, -3.7482667, 0.9256019, 2.1808605, -2.8636944, 2.688103, -0.62640125, 0.111900546, 0.39021575, -0.1782876, 0.33903104, -0.2657239, 1.8458211, -1.3479884, 0.99130964, 0.42136264, -0.29366258, -0.017333413, 0.75893915, -0.36884335, -0.2307046, 0.91578585, -0.908905, 0.28448543, 0.11103798, -0.19328305, -0.22101037, 0.09830263, -0.17263901, -0.2479087, -0.351095, -0.0387692, -0.783583, 'grey-cheeked warbler']

 Extracting Feature: 40 / 150 Files of the classs: grey-cheeked warbler
Path: ./reduce_sound/grey-cheeked warbler\phylloscopus poliogenys134.wav

Row added 40 
 
[-575.7319, -8.225976, 2.2706807, 4.0933576, -7.849049, 6.568322, -3.2723448, -0.5950411, 1.8082289, -2.6300557, 2.0165613, -0.9756256, 0.6883309, 0.3773489, -0.48765606, 1.057


 Extracting Feature: 52 / 150 Files of the classs: grey-cheeked warbler
Path: ./reduce_sound/grey-cheeked warbler\phylloscopus poliogenys145.wav

Row added 52 
 
[-628.5136, 1.6458888, -0.110210694, 1.2044739, -5.667893, 6.2438803, -0.057877228, -0.51528317, 0.27032703, -1.8843613, 2.3982716, 1.7526646, 0.70070636, -0.2224132, -0.17616099, 1.3083465, 1.1813736, 1.1704352, -0.10521833, 0.1279958, 0.55477005, 1.062888, 0.89366674, 0.24672161, 0.113591775, 0.33685455, 0.770361, 0.8237925, 0.26703778, 0.21486962, 0.1611281, 0.55939925, 0.40196785, 0.1694899, 0.1569071, 0.4014627, 0.47161546, 0.5377182, 0.2993782, 0.0005419465, 'grey-cheeked warbler']

 Extracting Feature: 53 / 150 Files of the classs: grey-cheeked warbler
Path: ./reduce_sound/grey-cheeked warbler\phylloscopus poliogenys146.wav

Row added 53 
 
[-685.68494, -23.108137, -2.9118767, 16.71587, -14.67331, 7.7825236, -1.0655487, -2.8440487, 2.6927984, -0.5033224, -0.21057057, -1.2226633, 1.0553057, 0.44236076, -0.049846157, -0.

...


 Extracting Feature: 67 / 132 Files of the classs: scaly laughingthrush
Path: ./reduce_sound/scaly laughingthrush\trochalopteron subunicolor40.wav

Row added 1285 
 
[-591.85956, -19.536758, -17.005928, 19.999655, -5.284153, -0.890746, 0.24091822, 5.0878177, -8.217312, 0.041458793, 0.2336209, -2.9576821, 1.7131227, -3.3459814, -4.4170237, -0.011834084, 1.6497203, -1.7929463, -0.11985898, -0.74111474, -0.8894686, 0.42907164, -0.04551855, -1.0350542, -0.4447402, 0.6845054, -1.2479677, 0.57676274, -0.8783427, -0.38528076, 0.2277843, -0.34862506, 1.0355966, 0.56892645, -0.24674065, 0.4966583, 0.006676205, 0.53103715, -0.10273703, 0.43421072, 'scaly laughingthrush']

 Extracting Feature: 68 / 132 Files of the classs: scaly laughingthrush
Path: ./reduce_sound/scaly laughingthrush\trochalopteron subunicolor41.wav

Row added 1286 
 
[-616.6666, -15.864664, -13.980169, 16.07824, -3.9799619, -1.1385517, -0.4894519, 4.0342884, -6.501161, -0.05846026, 0.5042467, -2.686816, 1.6345057, -2.5185826, 


 Extracting Feature: 80 / 132 Files of the classs: scaly laughingthrush
Path: ./reduce_sound/scaly laughingthrush\trochalopteron subunicolor52.wav

Row added 1298 
 
[-574.2092, 4.497869, -3.2471592, 16.465668, 7.052724, -2.5074546, 5.427745, 5.2205954, 0.22295316, 4.222279, 2.715127, 1.4782108, 2.7613769, 2.887242, -0.3473797, 3.6577938, 0.7812047, 1.7281841, 2.1042254, 1.4708339, 1.2830212, 1.9175767, 0.47586915, 1.4432911, 0.31756973, 1.6060615, 0.047378868, 0.53431046, 1.4589268, 0.23970309, 0.58355314, 1.4332666, -0.4783592, 1.2154056, 0.26878008, 0.8856476, 0.07514354, 1.2308615, 0.025132261, 0.8810777, 'scaly laughingthrush']

 Extracting Feature: 81 / 132 Files of the classs: scaly laughingthrush
Path: ./reduce_sound/scaly laughingthrush\trochalopteron subunicolor53.wav

Row added 1299 
 
[-592.17426, 1.4837612, -4.126655, 14.066421, 6.241441, -3.1833358, 4.9197774, 4.2033954, 0.39398625, 3.8890498, 2.4078958, 1.373637, 2.6665936, 2.9540954, -0.11179495, 2.9012084, 0.9722415, 


 Extracting Feature: 93 / 132 Files of the classs: scaly laughingthrush
Path: ./reduce_sound/scaly laughingthrush\trochalopteron subunicolor64.wav

Row added 1311 
 
[-565.596, -11.052616, -11.431827, 17.637941, 4.128102, -1.9605137, 3.3568652, 4.3697214, -0.99242115, 3.9588332, 4.8736315, -2.8598547, 3.2931619, 5.1183023, -1.3641553, 0.8030389, 3.3664372, 2.5633936, -1.2775836, 3.2868302, 0.7000931, 1.4990523, 1.8848425, 0.77969396, 1.5413479, 1.3571653, 1.2286478, 1.2788917, 1.1130416, 1.3543866, 1.2215804, 1.0512171, 1.1067444, 1.3605301, 0.6668555, 1.1422945, 1.4954282, 0.63224393, 0.54740596, 2.0107346, 'scaly laughingthrush']

 Extracting Feature: 94 / 132 Files of the classs: scaly laughingthrush
Path: ./reduce_sound/scaly laughingthrush\trochalopteron subunicolor65.wav

Row added 1312 
 
[-615.1986, -7.9273367, -7.9603434, 17.983828, 5.956607, 0.10692277, 5.090185, 6.1046453, 0.2941556, 5.260621, 6.3825326, -1.3440171, 4.476366, 5.9580193, -0.058772344, 1.9759976, 4.1309223, 3


 Extracting Feature: 106 / 132 Files of the classs: scaly laughingthrush
Path: ./reduce_sound/scaly laughingthrush\trochalopteron subunicolor76.wav

Row added 1324 
 
[-525.1403, -3.4435763, -6.562414, 6.79639, 0.6576065, -0.9438201, 1.7858278, 1.9041611, -1.40833, -0.29271543, 2.6795764, -0.74178094, 0.3670063, -0.17187695, -0.7714464, 0.56340396, 0.4054832, -0.31895855, -0.6442761, 0.7041556, 0.18306543, -0.44081816, 0.7513619, -0.3582485, -0.8909224, 0.027493937, 0.12762457, 0.26472238, -1.0155963, 0.36784193, -0.34599578, 0.35460734, -0.26846984, 0.038653973, -0.11502656, -0.1653386, -0.31569028, -0.040637683, -0.6816643, -0.15561439, 'scaly laughingthrush']

 Extracting Feature: 107 / 132 Files of the classs: scaly laughingthrush
Path: ./reduce_sound/scaly laughingthrush\trochalopteron subunicolor77.wav

Row added 1325 
 
[-547.8169, -3.47526, -5.407867, 5.50345, 0.6779069, -0.9968228, 1.3256657, 1.3806711, -1.2359174, -0.09511036, 2.3534088, -0.9614241, 0.3062864, 0.21801972, -0


 Extracting Feature: 119 / 132 Files of the classs: scaly laughingthrush
Path: ./reduce_sound/scaly laughingthrush\trochalopteron subunicolor88.wav

Row added 1337 
 
[-573.4889, -69.72291, -6.4901166, 33.56373, -34.87688, 21.233215, -7.821333, 2.735349, 3.8365543, -8.450535, 7.145592, -1.9596604, -4.602212, 7.528836, -6.4357443, 4.2775373, -1.980997, 0.04650186, 0.79482734, -0.67044884, 0.24647169, -0.4578145, 0.15535004, 0.42409483, -0.30088788, 0.31066585, -0.93321586, 1.3311421, -1.9021335, 2.1187887, -1.3234482, 0.499574, -0.29762226, 0.039977644, -0.038046688, 0.3167774, -0.15642104, 0.10748176, -0.3674879, 1.1311691, 'scaly laughingthrush']

 Extracting Feature: 120 / 132 Files of the classs: scaly laughingthrush
Path: ./reduce_sound/scaly laughingthrush\trochalopteron subunicolor89.wav

Row added 1338 
 
[-432.27582, -7.843652, -45.373573, -31.3128, -17.473036, 45.720135, -0.3089741, -17.241133, 2.858694, 6.950878, -15.679299, 3.5166655, 2.6713386, -1.1710765, -6.000511, 1.749

# converting extracted_features to Pandas dataframe

In [159]:
extracted_features_df=pd.DataFrame(final_df)
extracted_features_df

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,-472.100769,58.775963,7.681985,23.957186,0.592718,16.293337,2.327442,7.689467,4.085430,5.579339,...,3.295277,-1.296985,2.880492,-1.442738,2.679581,-0.661039,2.300287,-0.907895,2.506352,grey-cheeked warbler
1,-626.491150,12.133249,-13.319744,-1.620867,-5.934765,-5.031613,-9.316302,11.564653,-13.300512,1.906825,...,0.269395,0.246708,-1.039680,0.141618,-0.004299,0.990049,-0.811226,-0.073366,1.027817,grey-cheeked warbler
2,-469.611023,8.897367,-26.178530,33.399654,-20.362823,13.486960,-14.943768,-2.601473,-15.383101,-11.139963,...,-0.408060,1.530228,3.036365,0.097789,0.623564,3.737721,1.695220,0.724858,2.968561,grey-cheeked warbler
3,-831.966125,-10.816979,-0.287996,3.366931,-2.287159,9.823160,1.478227,3.595992,-0.649117,3.486054,...,2.860427,-1.280489,0.885871,-0.188271,1.080751,-0.926209,1.470803,-0.665033,1.119113,grey-cheeked warbler
4,-626.914001,12.200479,-12.821119,-2.286366,-4.333593,-5.938962,-8.716923,9.835328,-13.536802,1.394626,...,0.280611,0.247121,-0.828988,-0.126112,0.251314,0.786418,-0.675018,-0.024767,1.172768,grey-cheeked warbler
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,-647.261719,-12.537823,-19.363651,28.468985,-17.853638,1.917899,5.987606,0.248631,0.903875,-2.787823,...,0.868469,-0.053087,1.482946,-0.112607,0.619653,0.289150,0.543520,0.748008,0.304249,scaly laughingthrush
1346,-647.987915,-12.887599,-18.928745,28.989479,-17.074228,2.294021,6.127651,-0.009367,1.057003,-2.308768,...,0.901022,0.046337,1.367610,0.018777,0.665232,0.332188,0.594809,0.772941,0.317165,scaly laughingthrush
1347,-599.154419,-12.754396,-11.995712,20.265856,-2.581845,-2.186898,6.084940,2.563583,-0.376942,4.004078,...,2.116901,0.859350,0.748579,1.535090,1.793308,0.360066,1.238826,1.529079,0.289537,scaly laughingthrush
1348,-579.216675,6.736856,-4.974658,18.974611,8.814685,-5.747753,6.980567,4.489635,0.448849,4.898821,...,1.349982,-0.244144,1.016371,0.489593,0.491006,0.381481,0.981065,0.160588,0.736585,scaly laughingthrush


In [160]:
feature_length=len(extracted_features_df.columns)-1
#feature_length
feature_list = extracted_features_df.columns[:feature_length].tolist()
#feature_list

#feature DataSet
x=extracted_features_df[feature_list]
#x

#target DataSet
y=extracted_features_df[feature_length]
#y

In [161]:
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))
y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [162]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [163]:
num_labels=y.shape[1]
#Dense()
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.1))
###third layer
model.add(Dense(400))
model.add(Activation('relu'))
model.add(Dropout(0.1))
#fourth layer
model.add(Dense(600))
model.add(Activation('relu'))
model.add(Dropout(0.1))
###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [164]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 100)               4100      
_________________________________________________________________
activation_15 (Activation)   (None, 100)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 200)               20200     
_________________________________________________________________
activation_16 (Activation)   (None, 200)               0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 400)              

In [176]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
num_epochs =220
num_batch_size =32

checkpointer = ModelCheckpoint(filepath='audio_classification.hdf5',
    verbose=1,
    save_best_only=True)
start = datetime.now()

history = model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 1080 samples, validate on 270 samples
Epoch 1/220
1080/1080 [==============================] - ETA: 0s - loss: 0.1624 - acc: 0.9583
Epoch 00001: val_loss improved from inf to 1.43671, saving model to audio_classification.hdf5


`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


1080/1080 [==============================] - 0s 397us/sample - loss: 0.1624 - acc: 0.9583 - val_loss: 1.4367 - val_acc: 0.8556
Epoch 2/220
1024/1080 [===========================>..] - ETA: 0s - loss: 0.1040 - acc: 0.9688
Epoch 00002: val_loss improved from 1.43671 to 1.35935, saving model to audio_classification.hdf5
1080/1080 [==============================] - 0s 315us/sample - loss: 0.1023 - acc: 0.9685 - val_loss: 1.3594 - val_acc: 0.8815
Epoch 3/220
1056/1080 [============================>.] - ETA: 0s - loss: 0.0424 - acc: 0.9877
Epoch 00003: val_loss improved from 1.35935 to 1.30170, saving model to audio_classification.hdf5
1080/1080 [==============================] - 0s 306us/sample - loss: 0.0417 - acc: 0.9880 - val_loss: 1.3017 - val_acc: 0.8815
Epoch 4/220
 800/1080 [=====================>........] - ETA: 0s - loss: 0.0754 - acc: 0.9675
Epoch 00004: val_loss did not improve from 1.30170
1080/1080 [==============================] - 0s 153us/sample - loss: 0.0796 - acc: 0.9722 

Epoch 31/220
 704/1080 [==================>...........] - ETA: 0s - loss: 0.1235 - acc: 0.9773
Epoch 00031: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 85us/sample - loss: 0.1236 - acc: 0.9731 - val_loss: 1.0680 - val_acc: 0.8778
Epoch 32/220
 704/1080 [==================>...........] - ETA: 0s - loss: 0.1021 - acc: 0.9616
Epoch 00032: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 87us/sample - loss: 0.0940 - acc: 0.9620 - val_loss: 1.2847 - val_acc: 0.8704
Epoch 33/220
 704/1080 [==================>...........] - ETA: 0s - loss: 0.0544 - acc: 0.9815
Epoch 00033: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 86us/sample - loss: 0.0609 - acc: 0.9787 - val_loss: 1.3789 - val_acc: 0.8778
Epoch 34/220
 640/1080 [================>.............] - ETA: 0s - loss: 0.1936 - acc: 0.9750
Epoch 00034: val_loss did not improve from 0.98805
1080/1080 [===========================

 704/1080 [==================>...........] - ETA: 0s - loss: 0.0464 - acc: 0.9858
Epoch 00061: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 86us/sample - loss: 0.0437 - acc: 0.9870 - val_loss: 1.8253 - val_acc: 0.8444
Epoch 62/220
 608/1080 [===============>..............] - ETA: 0s - loss: 0.0715 - acc: 0.9770
Epoch 00062: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 93us/sample - loss: 0.0563 - acc: 0.9843 - val_loss: 1.4891 - val_acc: 0.8519
Epoch 63/220
 672/1080 [=================>............] - ETA: 0s - loss: 0.0515 - acc: 0.9807
Epoch 00063: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 87us/sample - loss: 0.0509 - acc: 0.9833 - val_loss: 1.5015 - val_acc: 0.8296
Epoch 64/220
 672/1080 [=================>............] - ETA: 0s - loss: 0.1201 - acc: 0.9673
Epoch 00064: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 86u

1080/1080 [==============================] - ETA: 0s - loss: 0.0808 - acc: 0.9731
Epoch 00091: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 107us/sample - loss: 0.0808 - acc: 0.9731 - val_loss: 1.2839 - val_acc: 0.8778
Epoch 92/220
 640/1080 [================>.............] - ETA: 0s - loss: 0.0293 - acc: 0.9859
Epoch 00092: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 89us/sample - loss: 0.0317 - acc: 0.9880 - val_loss: 1.3212 - val_acc: 0.8593
Epoch 93/220
 672/1080 [=================>............] - ETA: 0s - loss: 0.0895 - acc: 0.9732
Epoch 00093: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 87us/sample - loss: 0.0901 - acc: 0.9750 - val_loss: 1.6013 - val_acc: 0.8667
Epoch 94/220
 672/1080 [=================>............] - ETA: 0s - loss: 0.0819 - acc: 0.9792
Epoch 00094: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 87

 672/1080 [=================>............] - ETA: 0s - loss: 0.0644 - acc: 0.9851
Epoch 00121: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 88us/sample - loss: 0.0630 - acc: 0.9824 - val_loss: 1.0889 - val_acc: 0.8963
Epoch 122/220
 672/1080 [=================>............] - ETA: 0s - loss: 0.0498 - acc: 0.9807
Epoch 00122: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 89us/sample - loss: 0.0488 - acc: 0.9833 - val_loss: 1.2558 - val_acc: 0.8778
Epoch 123/220
 672/1080 [=================>............] - ETA: 0s - loss: 0.0347 - acc: 0.9881
Epoch 00123: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 88us/sample - loss: 0.0381 - acc: 0.9870 - val_loss: 1.3267 - val_acc: 0.8667
Epoch 124/220
 672/1080 [=================>............] - ETA: 0s - loss: 0.0564 - acc: 0.9866
Epoch 00124: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 

 672/1080 [=================>............] - ETA: 0s - loss: 0.0434 - acc: 0.9866    
Epoch 00151: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 88us/sample - loss: 0.0384 - acc: 0.9889 - val_loss: 1.4582 - val_acc: 0.8741
Epoch 152/220
 672/1080 [=================>............] - ETA: 0s - loss: 0.0375 - acc: 0.9911
Epoch 00152: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 94us/sample - loss: 0.0299 - acc: 0.9917 - val_loss: 1.4262 - val_acc: 0.8926
Epoch 153/220
 896/1080 [=======================>......] - ETA: 0s - loss: 0.0648 - acc: 0.9855
Epoch 00153: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 130us/sample - loss: 0.0688 - acc: 0.9815 - val_loss: 1.4014 - val_acc: 0.8741
Epoch 154/220
 896/1080 [=======================>......] - ETA: 0s - loss: 0.0625 - acc: 0.9833   
Epoch 00154: val_loss did not improve from 0.98805
1080/1080 [=============================

Epoch 181/220
 576/1080 [===============>..............] - ETA: 0s - loss: 0.0621 - acc: 0.9826
Epoch 00181: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 101us/sample - loss: 0.0628 - acc: 0.9815 - val_loss: 1.8287 - val_acc: 0.8741
Epoch 182/220
 576/1080 [===============>..............] - ETA: 0s - loss: 0.0413 - acc: 0.9878    
Epoch 00182: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 94us/sample - loss: 0.0718 - acc: 0.9833 - val_loss: 1.8575 - val_acc: 0.8704
Epoch 183/220
 672/1080 [=================>............] - ETA: 0s - loss: 0.0377 - acc: 0.9926
Epoch 00183: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 88us/sample - loss: 0.0328 - acc: 0.9926 - val_loss: 1.7227 - val_acc: 0.8704
Epoch 184/220
 672/1080 [=================>............] - ETA: 0s - loss: 0.0175 - acc: 0.9955
Epoch 00184: val_loss did not improve from 0.98805
1080/1080 [==================

Epoch 211/220
 608/1080 [===============>..............] - ETA: 0s - loss: 0.0640 - acc: 0.9819
Epoch 00211: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 96us/sample - loss: 0.0657 - acc: 0.9833 - val_loss: 1.6053 - val_acc: 0.8667
Epoch 212/220
 608/1080 [===============>..............] - ETA: 0s - loss: 0.0318 - acc: 0.9918
Epoch 00212: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 94us/sample - loss: 0.0534 - acc: 0.9861 - val_loss: 1.4907 - val_acc: 0.8667
Epoch 213/220
 608/1080 [===============>..............] - ETA: 0s - loss: 0.0791 - acc: 0.9786
Epoch 00213: val_loss did not improve from 0.98805
1080/1080 [==============================] - 0s 94us/sample - loss: 0.0993 - acc: 0.9713 - val_loss: 2.1132 - val_acc: 0.8593
Epoch 214/220
1024/1080 [===========================>..] - ETA: 0s - loss: 0.0687 - acc: 0.9785
Epoch 00214: val_loss did not improve from 0.98805
1080/1080 [=======================

In [177]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print('test loss:{} \ntest acc:{}'.format(test_accuracy[0],test_accuracy[1]))

test loss:1.425839270485772 
test acc:0.8481481671333313


In [178]:
#showing results and model accuracy 
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(num_epochs)

plt.figure(figsize=(12, 6))
plt.grid()

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()



validation_accuracy = np.mean(history.history['val_accuracy'])
accuracy = np.mean(history.history['accuracy'])

print('The overall Accuracy after traning: {}'.format(accuracy))

print('The overall Accuracy after Validation: {}'.format(validation_accuracy))

KeyError: 'accuracy'

In [ ]:
model.predict(X_test)

In [ ]:
filename="./Arunachal_Pradesh/indian spot-billed duck/pitch0.wav"
audio, sample_rate = librosa.load(filename,duration=5) 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)

predicted_label=np.argmax(model.predict(mfccs_scaled_features), axis=-1)

prediction_class = labelencoder.inverse_transform(predicted_label) 
print('The Pradictade Bird name for he given file is: {} '.format(prediction_class[0]))


In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTabularExplainer(X_train.values ,feature_names =feature_list ,
                                class_names =training_classes, mode = 'classification')    
exp = explainer.explain_instance(X_test.iloc[2], model.predict, num_features = 5)
exp.show_in_notebook(show_table = True, show_all = False)

In [ ]:
#load ML interpreting models
import shap

In [ ]:
#initialize JS for plot
shap.initjs()

In [ ]:
shap_explainer = shap.DeepExplainer(model,X_train[:100])
# **Compute shap_values for all of X_test rather instead of 
# a single row, to have more data for plot.**
shap_values = shap_explainer.shap_values(X_test[:10].values)

In [ ]:
# ## Variable Importance with Summary Plot

# In[ ]:


print("Variable Importance Plot - Global Interpretation")

shap.summary_plot(shap_values,X_test)

In [ ]:
# ## Summary Plot on a specific Label

# shap_values[0]
# shap_values[1]
shap.summary_plot(shap_values[0],X_test)
